In [ ]:
!pip install -q numpy pandas matplotlib scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

[Global Temperature Dataset (GitHub)](https://github.com/datasets/global-temp)
* Este repositório contém séries temporais mensais de anomalias de temperatura global desde 1850, com dados das fontes NOAA (GCAG) e NASA (GISTEMP).

In [ ]:
url = "https://raw.githubusercontent.com/datasets/global-temp/master/data/monthly.csv"
df = pd.read_csv(url)

df["date"] = pd.to_datetime(df["Year"])
df["value"] = df["Mean"].astype(np.float64)
df["Source"] = df["Source"].str.upper()

df = df[["Source", "date", "value"]]

df_gcag = df[df["Source"] == "GCAG"].drop("Source", axis=1).copy()
df_gistemp = df[df["Source"] == "GISTEMP"].drop("Source", axis=1).copy()

In [ ]:
def prever_anomalia(df, anos=100):
    df = df.copy()
    df = df.sort_values('date')
    df['timestamp'] = df['date'].astype('int64') // 10**9
    X = df[['timestamp']]
    y = df['value'].astype(np.float64)

    modelo = LinearRegression()
    modelo.fit(X, y)

    data_final = df['date'].max()
    datas_futuras = pd.date_range(start=data_final, periods=anos * 12, freq='ME')
    timestamps_futuros = datas_futuras.astype('int64') // 10**9
    X_futuro = pd.DataFrame({'timestamp': timestamps_futuros})

    previsoes = modelo.predict(X_futuro)
    df_pred = pd.DataFrame({'date': datas_futuras, 'value': previsoes.astype(np.float64)})

    return df_pred

In [ ]:
def plotar_previsao(df_real, df_pred, fonte):
    plt.figure(figsize=(12, 5))

    plt.plot(df_real['date'], df_real['value'], label=f'Histórico ({fonte})', color='orange')

    plt.plot(df_pred['date'], df_pred['value'], label=f'Previsão ({fonte})', linestyle='--', color='blue')

    plt.axhline(0, color='gray', linestyle='--', linewidth=1)
    
    plt.title(f'Histórico e Previsão de Anomalia de Temperatura ({fonte})')
    plt.xlabel('Ano')
    plt.ylabel('Δ Temperatura (°C)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
df_pred_gcag = prever_anomalia(df_gcag)
plotar_previsao(df_gcag, df_pred_gcag, 'GCAG')
df_pred_gistemp = prever_anomalia(df_gistemp)
plotar_previsao(df_gistemp, df_pred_gistemp, 'GISTEMP')